# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
from statistics import mean
from collections import Counter

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [2]:
with open('data/train-claims.json', 'r') as input_file:
    train_claim_data = json.load(input_file)

# Read in development data (claim)
with open('data/dev-claims.json', 'r') as input_file:
    dev_claim_data = json.load(input_file)

# Read in test data (claim)
with open('data/test-claims-unlabelled.json', 'r') as input_file:
    test_claim_data = json.load(input_file)

# Read in evidence data
with open('data/evidence.json', 'r') as input_file:
    evi_data = json.load(input_file)
    
#EDA




claim_count = 0
claim_length = []
evidence_count = []
evidence_length = []
labels = []

for key,value in train_claim_data.items():
    claim_count+=1
    claim_length.append(len(value["claim_text"]))
    evidence_count.append(len(value["evidences"]))
    evidence_length += [len(evi_data[x]) for x in value["evidences"]]
    labels.append(value["claim_label"])
    
print("claim count: ",claim_count)
print("max claim length: ",max(claim_length))
print("min claim length: ",min(claim_length))
print("mean claim length: ",mean(claim_length))
print("max evidence count: ",max(evidence_count))
print("min evidence count: ",min(evidence_count))
print("mean evidence count: ",mean(evidence_count))
print("max evidence length: ",max(evidence_length))
print("min evidence length: ",min(evidence_length))
print("mean evidence length: ",mean(evidence_length))
print(Counter(labels))



inside = 0
outside = 0

train_evi_id = []
for claim_id,claim_value in train_claim_data.items():
    train_evi_id=train_evi_id+claim_value['evidences']

for claim_id,claim_value in dev_claim_data.items():
    test_evi_id=claim_value['evidences']
    for e in test_evi_id:
        if e in train_evi_id:
            inside +=1
        else:
            outside += 1
print("Dev evi inside train evi", inside)
print("Dev evi outside train evi", outside)

full_evidence_id = list(evi_data.keys())
full_evidence_text  = list(evi_data.values())
train_claim_id = list(train_claim_data.keys())
train_claim_text  = [ v["claim_text"] for v in train_claim_data.values()]

claim count:  1228
max claim length:  332
min claim length:  26
mean claim length:  122.95521172638436
max evidence count:  5
min evidence count:  1
mean evidence count:  3.3566775244299674
max evidence length:  1979
min evidence length:  13
mean evidence length:  173.5
Counter({'SUPPORTS': 519, 'NOT_ENOUGH_INFO': 386, 'REFUTES': 199, 'DISPUTED': 124})
Dev evi inside train evi 163
Dev evi outside train evi 328


In [3]:

# dev remove stop word会好一点， test不remove反而好
evidence_tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)

# claim_tfidf_vectorizer = TfidfVectorizer(stop_words="english", use_idf=True)
claim_tfidf_vectorizer = TfidfVectorizer(use_idf=True)


evidence_tfidf_vectorizer.fit(train_claim_text+full_evidence_text)
train_claim_emb_list = claim_tfidf_vectorizer.fit_transform(train_claim_text)


full_evi_emb_list = evidence_tfidf_vectorizer.transform(full_evidence_text)


In [5]:

evi_k=2
claim_k=1

with open('data/dev-claims.json', 'r') as input_file:
    test_out_temp = json.load(input_file)

for claim_id,claim_value in test_out_temp.items():
    # Task1
    # 把test claim转化成vector
    
    test_claim_emb = evidence_tfidf_vectorizer.transform([claim_value['claim_text']])
    evi_sim_dict = {}
    # 计算出test claim和所有evidence的相似度
    sim = cosine_similarity(test_claim_emb, full_evi_emb_list)[0]
    
    for i in range(len(sim)):
        evi_sim_dict[full_evidence_id[i]] = sim[i]
    
    # 对evidence根据和claim的相似度排序
    s_sim = [(k, v) for k, v in sorted(evi_sim_dict.items(), key=lambda item: item[1],reverse=True)][:evi_k]
    sel_sim = [k for k,v in s_sim]
    # 把最相似的前k个evidence的id写入到test claim的evidence list
    test_out_temp[claim_id]["evidences"] = sel_sim
   
    # Task2
    # 把test claim转化成vector 注意我这两问用了不同的vectorizer，因为有不同的预处理步骤，和应用数据目标
    test_claim_emb = claim_tfidf_vectorizer.transform([claim_value['claim_text']])

    # 计算出test claim和所有train claim的相似度
    claim_sim_dict = {}
    claim_sim = cosine_similarity(test_claim_emb, train_claim_emb_list)[0]
    for i in range(len(claim_sim)):
        claim_sim_dict[train_claim_id[i]] = claim_sim[i]
    
    # 取最相似的k个train claim
    most_sim_claims = [(k, v) for k, v in sorted(claim_sim_dict.items(), key=lambda item: item[1],reverse=True)]
    # 我这里用的是k=1只考虑最相似的那一个，label拿出来
    most_sim_claim = max(most_sim_claims, key=operator.itemgetter(1))[0]
    
    test_out_temp[claim_id]["claim_label"] = train_claim_data[most_sim_claim]["claim_label"]


 
# Writing to sample.json
with open("data/dev_predict.json", "w") as outfile:
    json.dump(test_out_temp, outfile)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [6]:
# %%cmd
# python eval.py --predictions dev-claims-baseline.json --groundtruth dev-claims.json
# python eval.py --predictions dev_predict.json --groundtruth dev-claims.json


import subprocess

# proc = subprocess.Popen(["python", "eval.py", "--predictions", "data\dev_predict.json", "--groundtruth", "data\dev-claims.json"
# ], stdout=subprocess.PIPE, shell=True)
# (out, err) = proc.communicate()
# print(str(out))

# 高自动化模型/预处理选择，可以自动读取准确度
output = subprocess.check_output("python eval.py --predictions data/dev_predict.json --groundtruth data/dev-claims.json", shell=True)

print(output)

b'Evidence Retrieval F-score (F)    = 0.10510204081632651\r\nClaim Classification Accuracy (A) = 0.474025974025974\r\nHarmonic Mean of F and A          = 0.17205555936935077\r\n'


## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*